In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm

In [2]:
df_old=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/RegRelevant_ArticleSentimentScores.csv')
print(df_old.info())
print(df_old[['StartDate']])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502909 entries, 0 to 502908
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                502909 non-null  int64  
 1   StartDate         502909 non-null  object 
 2   Newspaper         502909 non-null  object 
 3   UncertaintyScore  502909 non-null  float64
 4   GIscore           502909 non-null  float64
 5   LMscore           502909 non-null  float64
 6   LSDscore          502909 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 26.9+ MB
None
         StartDate
0       1985-01-01
1       1985-01-01
2       1985-01-01
3       1985-01-01
4       1985-01-01
...            ...
502904  2020-08-31
502905  2020-08-31
502906  2020-08-31
502907  2020-08-31
502908  2020-08-31

[502909 rows x 1 columns]


In [3]:
df_new=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Update Data/RegRelevant_ArticleSentimentScores_Update.csv')
print(df_new.info())
print(df_new.sort_values('StartDate')[['StartDate']])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5832 entries, 0 to 5831
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                5832 non-null   int64  
 1   StartDate         5832 non-null   object 
 2   Newspaper         5832 non-null   object 
 3   UncertaintyScore  5832 non-null   float64
 4   GIscore           5832 non-null   float64
 5   LMscore           5832 non-null   float64
 6   LSDscore          5832 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 319.1+ KB
None
       StartDate
0     2020-09-01
3558  2020-09-01
3557  2020-09-01
3556  2020-09-01
3555  2020-09-01
...          ...
1040  2021-01-31
1041  2021-01-31
533   2021-01-31
2832  2021-01-31
1038  2021-01-31

[5832 rows x 1 columns]


In [4]:
df=df_old.append(df_new).sort_values(['StartDate','Newspaper'])
print(df.info())
print(df[['StartDate']])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508741 entries, 0 to 2834
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                508741 non-null  int64  
 1   StartDate         508741 non-null  object 
 2   Newspaper         508741 non-null  object 
 3   UncertaintyScore  508741 non-null  float64
 4   GIscore           508741 non-null  float64
 5   LMscore           508741 non-null  float64
 6   LSDscore          508741 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 31.1+ MB
None
       StartDate
0     1985-01-01
1     1985-01-01
2     1985-01-01
3     1985-01-01
4     1985-01-01
...          ...
2830  2021-01-31
2831  2021-01-31
2832  2021-01-31
2833  2021-01-31
2834  2021-01-31

[508741 rows x 1 columns]


In [5]:
df['StartDate']=df['StartDate'].astype('datetime64[ns]')
df['Year']=df['StartDate'].dt.year
df['Month']=df['StartDate'].dt.month
df['Newspaper']=df['Newspaper'].astype('category')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508741 entries, 0 to 2834
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                508741 non-null  int64         
 1   StartDate         508741 non-null  datetime64[ns]
 2   Newspaper         508741 non-null  category      
 3   UncertaintyScore  508741 non-null  float64       
 4   GIscore           508741 non-null  float64       
 5   LMscore           508741 non-null  float64       
 6   LSDscore          508741 non-null  float64       
 7   Year              508741 non-null  int64         
 8   Month             508741 non-null  int64         
dtypes: category(1), datetime64[ns](1), float64(4), int64(3)
memory usage: 35.4 MB
None


In [6]:
df_ym=df[['Year','Month']].drop_duplicates().reset_index(drop=True).reset_index()
df_ym['YM']=df_ym['index']+1
df_ym=df_ym.drop('index',axis=1)
print(df_ym.tail())

     Year  Month   YM
428  2020      9  429
429  2020     10  430
430  2020     11  431
431  2020     12  432
432  2021      1  433


In [7]:
df=df.merge(df_ym[['Year','Month','YM']],on=['Year','Month'],how='left')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508741 entries, 0 to 508740
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                508741 non-null  int64         
 1   StartDate         508741 non-null  datetime64[ns]
 2   Newspaper         508741 non-null  category      
 3   UncertaintyScore  508741 non-null  float64       
 4   GIscore           508741 non-null  float64       
 5   LMscore           508741 non-null  float64       
 6   LSDscore          508741 non-null  float64       
 7   Year              508741 non-null  int64         
 8   Month             508741 non-null  int64         
 9   YM                508741 non-null  int64         
dtypes: category(1), datetime64[ns](1), float64(4), int64(4)
memory usage: 39.3 MB
None


In [8]:
print(df[['ID','UncertaintyScore','GIscore','LMscore','LSDscore']].head())

          ID  UncertaintyScore   GIscore   LMscore  LSDscore
0  294326637          0.000000  0.000000 -2.631579  0.000000
1  294308147          0.000000  4.210526 -1.052632  0.000000
2  294323196          1.428571  2.857143  0.000000 -1.428571
3  294262284          0.938967  2.347418 -1.408451  0.000000
4  294234319          0.000000  1.449275  0.000000  0.000000


In [9]:
print(df['Newspaper'].value_counts())

Wall Street Journal    169229
New York Times          74363
Los Angeles Times       74221
The Washington Post     72262
Chicago Tribune         52235
Boston Globe            44842
USA Today               21589
Name: Newspaper, dtype: int64


In [10]:
# Revised function to estimate index (suppressing constant)
def estimate_index(var_name):
    FE_OLS=sm.ols(formula=var_name + ' ~ 0+C(YM)+C(Newspaper)',
        data=df).fit()
    print(FE_OLS.summary())

    FE_estimates=pd.DataFrame()
    FE_estimates[var_name+'Index']=FE_OLS.params[0:max(df_ym['YM'])]
    FE_estimates=FE_estimates.reset_index().rename(columns={'index':'FE'})
    FE_estimates['YM']=FE_estimates['FE'].str.split("[",expand=True)[1].str.split("]",expand=True)[0].astype('int64')
    
    return FE_estimates

In [11]:
# Uncertainty index
UncertaintyIndex=estimate_index('UncertaintyScore')

                            OLS Regression Results                            
Dep. Variable:       UncertaintyScore   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     6.005
Date:                Mon, 01 Feb 2021   Prob (F-statistic):          1.64e-307
Time:                        20:33:12   Log-Likelihood:            -6.9233e+05
No. Observations:              508741   AIC:                         1.386e+06
Df Residuals:                  508302   BIC:                         1.390e+06
Df Model:                         438                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
C(

In [12]:
# LM index
LMindex=estimate_index('LMscore')

                            OLS Regression Results                            
Dep. Variable:                LMscore   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     7.088
Date:                Mon, 01 Feb 2021   Prob (F-statistic):               0.00
Time:                        20:34:07   Log-Likelihood:            -1.2019e+06
No. Observations:              508741   AIC:                         2.405e+06
Df Residuals:                  508302   BIC:                         2.409e+06
Df Model:                         438                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
C(

In [13]:
# GI index
GIindex=estimate_index('GIscore')

                            OLS Regression Results                            
Dep. Variable:                GIscore   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     6.360
Date:                Mon, 01 Feb 2021   Prob (F-statistic):               0.00
Time:                        20:35:03   Log-Likelihood:            -1.4250e+06
No. Observations:              508741   AIC:                         2.851e+06
Df Residuals:                  508302   BIC:                         2.856e+06
Df Model:                         438                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
C(

In [14]:
# LSD index
LSDindex=estimate_index('LSDscore')

                            OLS Regression Results                            
Dep. Variable:               LSDscore   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     7.423
Date:                Mon, 01 Feb 2021   Prob (F-statistic):               0.00
Time:                        20:35:45   Log-Likelihood:            -1.3469e+06
No. Observations:              508741   AIC:                         2.695e+06
Df Residuals:                  508302   BIC:                         2.700e+06
Df Model:                         438                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
C(

In [15]:
# Merge indexes
sentimentIndex=df_ym.merge(UncertaintyIndex,on='YM',how='outer').\
        merge(LMindex,on='YM',how='outer').\
        merge(GIindex,on='YM',how='outer').\
        merge(LSDindex,on='YM',how='outer').\
        sort_values(['Year','Month'])
print(sentimentIndex.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 433 entries, 0 to 432
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year                   433 non-null    int64  
 1   Month                  433 non-null    int64  
 2   YM                     433 non-null    int64  
 3   FE_x                   433 non-null    object 
 4   UncertaintyScoreIndex  433 non-null    float64
 5   FE_y                   433 non-null    object 
 6   LMscoreIndex           433 non-null    float64
 7   FE_x                   433 non-null    object 
 8   GIscoreIndex           433 non-null    float64
 9   FE_y                   433 non-null    object 
 10  LSDscoreIndex          433 non-null    float64
dtypes: float64(4), int64(3), object(4)
memory usage: 40.6+ KB
None


In [16]:
sentimentIndex=sentimentIndex.drop(['FE_x','FE_y'],axis=1).\
        rename(columns={'UncertaintyScoreIndex':'UncertaintyIndex','LMscoreIndex':'LMindex',
                        'GIscoreIndex':'GIindex','LSDscoreIndex':'LSDindex'})

In [17]:
print(sentimentIndex.head())

   Year  Month  YM  UncertaintyIndex   LMindex   GIindex  LSDindex
0  1985      1   1          0.606447 -1.797994  1.352665  0.255111
1  1985      2   2          0.592279 -1.905220  1.483960  0.060418
2  1985      3   3          0.628521 -1.966936  1.217096 -0.045179
3  1985      4   4          0.647759 -1.848431  1.422165  0.085410
4  1985      5   5          0.653402 -1.779092  1.460020  0.280421


In [18]:
# # Set the first value as zero
# for col in ['UncertaintyIndex','LMindex','GIindex','LSDindex']:
#     sentimentIndex.loc[sentimentIndex[col].isnull(),col]=0
# print(sentimentIndex.head())

In [19]:
print(sentimentIndex.info())
print(sentimentIndex.tail())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 433 entries, 0 to 432
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              433 non-null    int64  
 1   Month             433 non-null    int64  
 2   YM                433 non-null    int64  
 3   UncertaintyIndex  433 non-null    float64
 4   LMindex           433 non-null    float64
 5   GIindex           433 non-null    float64
 6   LSDindex          433 non-null    float64
dtypes: float64(4), int64(3)
memory usage: 27.1 KB
None
     Year  Month   YM  UncertaintyIndex   LMindex   GIindex  LSDindex
428  2020      9  429          0.669940 -1.893831  1.637110  0.612628
429  2020     10  430          0.717963 -1.740618  1.365803  0.288503
430  2020     11  431          0.755689 -1.520050  1.535037  0.578050
431  2020     12  432          0.631321 -1.651442  1.275530  0.411107
432  2021      1  433          0.671965 -1.583927  1.690167  0.5201

In [20]:
# Export
sentimentIndex.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Update Data/RegRelevant_MonthlySentimentIndex_Update.csv',index=False)